In [1]:
val bucketname = "unibo-bigdata-project"

val pathSales = "s3a://"+bucketname+"/datasets/Liquor_Sales.csv"

sc.applicationId

"SPARK UI: Enable forwarding of port 20888 and connect to http://localhost:20888/proxy/" + sc.applicationId + "/"

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
0,application_1678372684627_0001,spark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

bucketname: String = unibo-bigdata-project
pathSales: String = s3a://unibo-bigdata-project/datasets/Liquor_Sales.csv
res3: String = application_1678372684627_0001
res5: String = SPARK UI: Enable forwarding of port 20888 and connect to http://localhost:20888/proxy/application_1678372684627_0001/


In [2]:
import org.apache.spark.sql.SaveMode

object LiquorSalesParser {
    


  
  /** Function to parse stores records
   *
   *  @param line line that has to be parsed
   *  @return tuple containing storeNumber, storeName,, productsID none in case of input errors
   */
    
  def parseStores(line: String) : Option[(Long, String, String)] = {
    try {
      val input = line.split(',')
      Some(input(2).trim.toLong, input(3),  input(16))
    } catch {
      case _: Exception => None
    }
  }  
    
    
    
  /** Function to parse items (products) records
   *
   *  @param line line that has to be parsed
   *  @return tuple containing itemNumber, itemName, Sales($), categoryName none in case of input errors
   */
    
    def parseItems(line: String) : Option[(Long, String, Double, String, Int)] = {
      try {
        val input = line.split(',')
        val date = input(1)
        val year = date.split("/").last.toInt
        Some(input(14).trim.toLong, input(15), input(22).trim.toDouble, input(11), year)
      } catch {
        case _: Exception => None
      }
    }
 


/** Function to parse country records
   *
   *  @param line line that has to be parsed
   *  @return tuple containing countryNumber, countryName, VolumeSold(L), Sales($), date
   */
    
  def parseCountries(line: String) : Option[(String, String, Double, Double, String)] = {
    try {
      val input = line.split(',')
      Some(input(9), input(10), input(23).trim.toDouble, input(22).trim.toDouble, input(2))
    } catch {
      case _: Exception => None
    }
  }  
    
    
/** Function to parse city records
   *
   *  @param line line that has to be parsed
   *  @return tuple containing zipCode, cityName
   */
    
  def parseCities(line: String) : Option[(String, Long)] = {
    try {
      val input = line.split(',')
      Some(input(6), input(7).trim.toLong)
    } catch {
      case _: Exception => None
    }
  }

    
    
/** Function to parse categories records
   *
   *  @param line line that has to be parsed
   *  @return tuple containing categoryNumber, categoryName, bottleSold,  Sales($)
   */
    
  def parseCategories(line: String) : Option[(Long, String, Long, Double)] = {
    try {
      val input = line.split(',')
      Some(input(11).trim.toLong, input(12), input(21).trim.toLong, input(22).trim.toDouble)
    } catch {
      case _: Exception => None
    }
  }
    
    
/** Function to parse vendors records
   *
   *  @param line line that has to be parsed
   *  @return tuple containing vendorNumber, vendorName
   */
    
  def parseVendors(line: String) : Option[( Long, String)] = {
    try {
      val input = line.split(',')
      Some(input(13).trim.toLong, input(14))
    } catch {
      case _: Exception => None
    }
  }    
}

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

import org.apache.spark.sql.SaveMode
defined object LiquorSalesParser


In [3]:
val rddStores = sc.textFile(pathSales).flatMap(LiquorSalesParser.parseStores)
val rddItems = sc.textFile(pathSales).flatMap(LiquorSalesParser.parseItems)
val rddCountries = sc.textFile(pathSales).flatMap(LiquorSalesParser.parseCountries)
val rddCities = sc.textFile(pathSales).flatMap(LiquorSalesParser.parseCities)
val rddCategories = sc.textFile(pathSales).flatMap(LiquorSalesParser.parseCategories)
val rddVendors = sc.textFile(pathSales).flatMap(LiquorSalesParser.parseVendors)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

rddStores: org.apache.spark.rdd.RDD[(Long, String, String)] = MapPartitionsRDD[2] at flatMap at <console>:26
rddItems: org.apache.spark.rdd.RDD[(Long, String, Double, String, Int)] = MapPartitionsRDD[5] at flatMap at <console>:26
rddCountries: org.apache.spark.rdd.RDD[(String, String, Double, Double, String)] = MapPartitionsRDD[8] at flatMap at <console>:26
rddCities: org.apache.spark.rdd.RDD[(String, Long)] = MapPartitionsRDD[11] at flatMap at <console>:26
rddCategories: org.apache.spark.rdd.RDD[(Long, String, Long, Double)] = MapPartitionsRDD[14] at flatMap at <console>:26
rddVendors: org.apache.spark.rdd.RDD[(Long, String)] = MapPartitionsRDD[17] at flatMap at <console>:26


In [4]:
/*Da questa analisi possiamo capire che il numero degli stores number e degli stores name è diverso => che il nome di un negozio può
essere cambiato nel corso degli anni => negozio#1(nome1,nome2,nome3)*/

val rddStoresCached = rddStores.cache()
"Number of Stores group by their name: " + rddStoresCached.map(x => (x._2)).distinct().count()
"Number of Stores group by their code: " + rddStoresCached.map(x => (x._1)).distinct().count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

rddStoresCached: rddStores.type = MapPartitionsRDD[2] at flatMap at <console>:26
res9: String = Number of Stores group by their name: 2632
res10: String = Number of Stores group by their code: 2484


In [5]:
/*Da questa analisi possiamo capire che il numero degli items number e degli items name è diverso*/

val rddItemsCached = rddItems.cache()
"Number of Items group by their name: " + rddItemsCached.map(x => (x._2)).distinct().count()
"Number of Items group by their code: " + rddItemsCached.map(x => (x._1)).distinct().count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

rddItemsCached: rddItems.type = MapPartitionsRDD[5] at flatMap at <console>:26
res13: String = Number of Items group by their name: 7834
res14: String = Number of Items group by their code: 8322


In [6]:
val rddCountriesCached = rddCountries.cache()
"Number of countries group by their name: " + rddCountriesCached.map(x => (x._2)).distinct().count()
"Number of countries group by their code: " + rddCountriesCached.map(x => (x._1)).distinct().count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

rddCountriesCached: rddCountries.type = MapPartitionsRDD[8] at flatMap at <console>:26
res15: String = Number of countries group by their name: 277
res16: String = Number of countries group by their code: 298


In [7]:
val rddCitiesCached = rddCities.cache()
"Number of cities group by their name: " + rddCitiesCached.map(x => (x._2)).distinct().count()
"Number of cities group by their code: " + rddCitiesCached.map(x => (x._1)).distinct().count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

rddCitiesCached: rddCities.type = MapPartitionsRDD[11] at flatMap at <console>:26
res17: String = Number of cities group by their name: 218
res18: String = Number of cities group by their code: 354


In [8]:
val rddCategoriesCached = rddCategories.cache()
"Number of categories group by their name: " + rddCategoriesCached.map(x => (x._2)).distinct().count()
"Number of categories group by their code: " + rddCategoriesCached.map(x => (x._1)).distinct().count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

rddCategoriesCached: rddCategories.type = MapPartitionsRDD[14] at flatMap at <console>:26
res19: String = Number of categories group by their name: 128
res20: String = Number of categories group by their code: 100


In [9]:
val rddVendorsCached = rddVendors.cache()
"Number of vendors group by their name: " + rddVendorsCached.map(x => (x._2)).distinct().count()
"Number of vendors group by their code: " + rddVendorsCached.map(x => (x._1)).distinct().count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

rddVendorsCached: rddVendors.type = MapPartitionsRDD[17] at flatMap at <console>:26
res21: String = Number of vendors group by their name: 377
res22: String = Number of vendors group by their code: 262


In [10]:
/*In Media, quanti prodotti sono stati venduti per negozio?*/
val rddStoresCached = rddStores.cache()
val avgProductSoldPerShop = rddStoresCached.map(x => (x._1,1)).reduceByKey(_+_).aggregate((0,0))((a,v)=>(a._1+v._2, a._2+1),(a1,a2)=>(a1._1+a2._1,a1._2+a2._2))
"Average number of products sold per shop: " + (avgProductSoldPerShop._1/avgProductSoldPerShop._2)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

rddStoresCached: rddStores.type = MapPartitionsRDD[2] at flatMap at <console>:26
avgProductSoldPerShop: (Int, Int) = (19666763,2484)
res24: String = Average number of products sold per shop: 7917


In [11]:
/*In Media, quanti litri sono stati vednuti per stato?*/

val rddCountiresCached = rddCountries.cache()
val avgLitersSoldPerCountry = rddCountries.
    map(x => (x._1,x._3)). /*(countryNumber,liters)*/
    countByKey().
    aggregate((0.0,0.0))((a,v)=>(a._1+v._2, a._2+1),(a1,a2)=>(a1._1+a2._1,a1._2+a2._2))
"Media di litri venduti per country: " + (avgLitersSoldPerCountry._1/avgLitersSoldPerCountry._2)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

rddCountiresCached: rddCountries.type = MapPartitionsRDD[8] at flatMap at <console>:26
avgLitersSoldPerCountry: (Double, Double) = (1.9666762E7,298.0)
res27: String = Media di litri venduti per country: 65995.84563758389


In [12]:
/*In Media, quante bottiglie sono state vednute per categoria?*/

val rddCategoriesCached = rddCategories.cache()
val avgBottlesPerCategory = rddCategories.
    map(x => (x._1,x._3)).
    countByKey().
    aggregate((0.0,0.0))((a,v)=>(a._1+v._2, a._2+1),(a1,a2)=>(a1._1+a2._1,a1._2+a2._2))
"Media di bottiglie vendute per categoria: " + (avgBottlesPerCategory._1/avgBottlesPerCategory._2)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

rddCategoriesCached: rddCategories.type = MapPartitionsRDD[14] at flatMap at <console>:26
avgBottlesPerCategory: (Double, Double) = (1559988.0,100.0)
res30: String = Media di bottiglie vendute per categoria: 15599.88


In [13]:
/*items più venduti in termini di fatturato e unità*/
println("Risultato cosi' composto: nomeItem, fatturato, numero di vendite \n")
val rddItemsCached = rddItems.cache()
rddItemsCached.
        map(e => (e._2, (e._3, 1))).
        reduceByKey((t1, t2) => (t1._1 + t2._1, t1._2 + t2._2)).
        map(v => (v._1, v._2._1, v._2._2)).
        sortBy(_._2, false).
        collect().take(10).foreach(println(_))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Risultato cosi' composto: nomeItem, fatturato, numero di vendite 

rddItemsCached: rddItems.type = MapPartitionsRDD[5] at flatMap at <console>:26
(Hawkeye Vodka,7275708.29,411211)
(Black Velvet,5242031.529999999,220957)
(Captain Morgan Spiced Rum,4018092.1699999985,188778)
(Titos Handmade Vodka,3770229.4600000014,166741)
(Barton Vodka,3219200.8399999994,112894)
(Five O'clock Vodka,2997823.2799999984,226502)
(Phillips Vodka,2366347.33,107766)
(Jack Daniels Old #7 Black Lbl,2306382.9500000007,159353)
(Admiral Nelson Spiced Rum,1944860.9900000014,177337)
(Fireball Cinnamon Whiskey,1850195.199999996,122337)


In [9]:
/*Categorie più vendute in termini di fatturato e unità*/
println("Risultato cosi' composto: nomeCategor, fatturato, numero di vendite \n")
val rddCategoriesCached = rddCategories.cache()
rddCategoriesCached.
        map(e => (e._2, (e._4, 1))).
        reduceByKey((t1, t2) => (t1._1 + t2._1, t1._2 + t2._2)).
        map(v => (v._1, v._2._1, v._2._2)).
        sortBy(_._2, false).
        collect().take(10).foreach(println(_))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Risultato cosi' composto: nomeCategor, fatturato, numero di vendite 

rddCategoriesCached: rddCategories.type = MapPartitionsRDD[14] at flatMap at <console>:26
(American Vodkas,1.4152316429999966E7,140485)
(Canadian Whiskies,1.389965760000001E7,103166)
(VODKA 80 PROOF,9530538.619999986,109531)
(Imported Brandies,8827292.330000015,30078)
(Spiced Rum,7252489.699999989,49083)
(SPICED RUM,7068700.709999999,44743)
(Whiskey Liqueur,6887928.550000002,50702)
(CANADIAN WHISKIES,6197296.109999994,47384)
(TEQUILA,4861197.939999997,35800)
(Straight Bourbon Whiskies,4779353.42,47715)


In [11]:
/*val bucketname = "unibo-bigdata-project"
val path_categories_revenue = "s3a://"+bucketname+"/risultati/satates_revenue"*/

val rddItemsCached = rddItems.cache()
rddItemsCached.
        map(e => (e._2, (e._3, 1))). //nameItems,(price,1)
        reduceByKey((t1, t2) => (t1._1 + t2._1, t1._2 + t2._2)). //nameItem(fatturato,unitàVendute)
        map(v => (v._1, v._2._1, v._2._2)). //nomeItem, fatturato, unitàVendute
        sortBy(_._2, false)


val itemsClassification = rddItemsCached.
    map(e => (e._2, e._3)). // nomeItem, price
    reduceByKey(_ + _).
    map(e => (e._1, e._2 match {
      case f if f >= 1000000 => "Top Tier"
      case f if f < 1000000 && f >= 100000 => "Mid Tier"  
      case _ => "Low Tier"
      })) //nomeItem, Tier


val revenueByCategory = rddItemsCached.
    map(p => (p._2, (p._3, p._4))). // nomeItem, (price, categoryName)
    join(itemsClassification). // nomeItem, ((price,categoryName), itemRank)
    flatMap(p => p._2._1._2.map(x => ((x, p._2._2), p._2._1._1))). // (categoryName, itemRank), price
    aggregateByKey((0.0,0))((agg, v) => (agg._1 + v, agg._2 + 1),((agg1,agg2) => (agg1._1 + agg2._1, agg1._2 + agg2._2))). // (categoryName, itemRank), price, count
    map(x => (x._1._1, x._1._2, x._2._1, x._2._2)). // categoryName, itemRank, revenue, count
    collect()

/*val revenueByCategoryDF = revenueByCategory.toDF()
revenueByCategoryDF.coalesce(1).write.format("csv").mode(SaveMode.Overwrite).save(path_categories_revenue)*/

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

rddItemsCached: rddItems.type = MapPartitionsRDD[5] at flatMap at <console>:26
res66: org.apache.spark.rdd.RDD[(String, Double, Int)] = MapPartitionsRDD[125] at sortBy at <console>:29
itemsClassification: org.apache.spark.rdd.RDD[(String, String)] = MapPartitionsRDD[128] at map at <console>:29
revenueByCategory: Array[(Char, String, Double, Int)] = Array((b,Mid Tier,1795481.2000000014,201407), (h,Top Tier,1.476467419000033E7,1033752), (M,Low Tier,8189084.839999994,1729963), (W,Low Tier,7484470.299999987,1660244), (O,Mid Tier,3.1257047840003908E7,3167448), (G,Top Tier,1816253.629999969,180109), (v,Low Tier,4190338.6400000006,801962), (d,Low Tier,1.5083578329999989E7,2963354), (q,Top Tier,2153148.300000113,154716), (l,Mid Tier,9470410.019999977,728839), (L,Low Tier,9508378.559999932,2083313), (P,Mid Tier,1.696546511000002E7,1750418), (K,Mid Tier,1.5498019640000032E7,1586833), (I,Mid Tier,4.666476288000064E7,4785276), (f,Low Tier,118806.05999999994,34912), (C,Top Tier,1.7752091380000554E7

In [8]:
val bucketname = "unibo-bigdata-project"
val path_itemsForYear_revenue = "s3a://"+bucketname+"/risultati/satates_revenue"

// calcolare il prezzo medio di ogni prodotto per ogni anno
val rddItemsCached = rddItems.cache()

val avgPricesByYear = rddItems.map(item => ((item._2, item._5), (item._3, 1))).//(itemName,anno), price
  reduceByKey((accum, value) => (accum._1 + value._1, accum._2 + value._2)).
  mapValues(sumCount => sumCount._1 / sumCount._2).
  map(item => (item._1._1, item._2, item._1._2))
  //collect() //(itemName,avg,year)

// Raggruppo per nomeItem
val avgPricesByProduct = avgPricesByYear.groupBy(_._1).
  mapValues(_.map(item => (item._2, item._3)))//.collect()

//flatmap con chiave prodotto e anno
val avgPricesByProductAndYear = avgPricesByProduct.flatMap { case (itemName, yearPrices) =>
  yearPrices.map { case (avgPrice, year) =>
    ((itemName, year), avgPrice)
  }
}.coalesce(1)

// Conversione in DataFrame con stringa
val df = avgPricesByProductAndYear.map{case ((itemName, year), avgPrice) => (itemName,year,avgPrice)}.toDF()

// Salvataggio su file CSV
df.write.format("csv").mode(SaveMode.Overwrite).save(path_itemsForYear_revenue)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

bucketname: String = unibo-bigdata-project
path_itemsForYear_revenue: String = s3a://unibo-bigdata-project/risultati/satates_revenue
rddItemsCached: rddItems.type = MapPartitionsRDD[5] at flatMap at <console>:26
avgPricesByYear: org.apache.spark.rdd.RDD[(String, Double, Int)] = MapPartitionsRDD[75] at map at <console>:28
avgPricesByProduct: org.apache.spark.rdd.RDD[(String, Iterable[(Double, Int)])] = MapPartitionsRDD[78] at mapValues at <console>:28
avgPricesByProductAndYear: org.apache.spark.rdd.RDD[((String, Int), Double)] = CoalescedRDD[80] at coalesce at <console>:30
df: org.apache.spark.sql.DataFrame = [_1: string, _2: int ... 1 more field]


In [7]:
val bucketname = "unibo-bigdata-project"
val path_category_revenue = "s3a://"+bucketname+"/risultati/categories_revenue"

val rddItemsCached = rddItems.cache()
val v = rddItemsCached.
        map(e => ((e._2, e._4), (e._3, 1))). //(nameItem, categoryName),(price,1)
        reduceByKey((t1, t2) => (t1._1 + t2._1, t1._2 + t2._2)). //(nameItem,categoryName),(fatturato,unitàVendute)
        map(v => (v._1._1, v._1._2, v._2._1, v._2._2)). //nomeItem, categoryName, fatturato, unitàVendute
        sortBy(_._3, false)


val itemsClassification = v.
    map(e => (e._1, e._3)). // nomeItem, fatturato
    reduceByKey(_ + _).
    map(e => (e._1, e._2 match {
      case f if f >= 1000000 => "Top Tier"
      case f if f < 1000000 && f >= 100000 => "Mid Tier"  
      case _ => "Low Tier"
      })) //nomeItem, Tier


val revenueByCategory = v.
    map(p => (p._1, (p._3, p._2))). // nomeItem, (fatturato, categoryName)
    join(itemsClassification). // nomeItem, ((fatturato,categoryName), Tier)
    map(p => ((p._2._1._2, p._2._2), p._2._1._1)). // (categoryName, Tier), fatturato
    aggregateByKey((0.0,0))((agg, v) => (agg._1 + v, agg._2 + 1),((agg1,agg2) => (agg1._1 + agg2._1, agg1._2 + agg2._2))). // (categoryName, Tier), price, count
    map(x => (x._1._1, x._1._2, x._2._1, x._2._2)).// categoryName, Tier, revenue, count
    coalesce(1)

revenueByCategory.map(x => x.productIterator.mkString(",")).saveAsTextFile(path_category_revenue)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

bucketname: String = unibo-bigdata-project
path_category_revenue: String = s3a://unibo-bigdata-project/risultati/categories_revenue
rddItemsCached: rddItems.type = MapPartitionsRDD[5] at flatMap at <console>:26
v: org.apache.spark.rdd.RDD[(String, String, Double, Int)] = MapPartitionsRDD[58] at sortBy at <console>:29
itemsClassification: org.apache.spark.rdd.RDD[(String, String)] = MapPartitionsRDD[61] at map at <console>:29
revenueByCategory: org.apache.spark.rdd.RDD[(String, String, Double, Int)] = CoalescedRDD[69] at coalesce at <console>:33
